# Exercise 1 - Part 1 -  Protein in Water

You can run this example directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm_workshop/blob/main/exercise_1/exercise_1_part1.ipynb)


## Introduction

We will cover the following steps:
- Setup conda environment
- Load a PDB file into OpenMM
- Choose the force-field
- Solvate the protein with water and ions
- Setup system and integrator
- Run local minimization
- Run NVT equilibration
- Run NPT production molecular dynamics
- Basic analysis
- How to use checkpoints


## Setup

If you are using Colab you can run the cell below to install mamba in the Colab environment.

**First try and change runtime type to GPU!**

It will work on CPU but will be slower.

If you want to run on your own machine take a look at the [setup instructions](https://github.com/sef43/openmm_workshop/tree/main/setup). 
Remember you can replace `mamba` with `conda` if you have not installed `mamba`.

In [ ]:
# Execute this cell to install mamba in the Colab environment

if 'google.colab' in str(get_ipython()):
  print('Running on colab')
  !pip install -q condacolab
  import condacolab
  condacolab.install_mambaforge()
else:
  print('Not running on colab.')
  print('Make sure you create and activate a new conda environment!')

Now we can install openmm from conda-forge

In [ ]:
!mamba install -y -c conda-forge openmm

Test the installation

In [ ]:
!python -m openmm.testInstallation

## Download the protein structure file

We will download the file from the workshop github repo

In [ ]:
!wget https://raw.githubusercontent.com/sef43/openmm_workshop/main/exercise_1/villin.pdb

The protein is the villin headpiece. This is small fast folding protein commonly used as a toy system.

![villin](villin.png)
**Figure**. Villin headpiece protein.

## Load the PDB file into OpenMM

First we need to import OpenMM.
We then then load in the PDB file/

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

# load in the pdb file
pdb = PDBFile('villin.pdb')

## Define the forcefield

We need to define the forcefield we want to use. We will use the Amber14 forcefield and the TIP3P-FB water model.

In [ ]:
# Specify the forcefield
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')


## Solvate

We can use the `Modeller` class to solvate the protein in waterbox.

In [ ]:
# create a Modeller object
modeller = Modeller(pdb.topology, pdb.positions)


# Solvate the protein in a box of water
modeller.addSolvent(forcefield, padding=1.0*nanometer)

This command creates a box that has edges at least 1nm away from the solute and fills it with water molecules. Additionally it adds in the required number of CL and Na ions to make the system charge neutral.

## Setup system and Integrator

We now need to combine our molecular topology and the forcefield to create a complete description of the system. This is done using the `ForceField` object’s `createSystem()` function. We then create the integrator, and combine the integrator and system to create the Simulation object. Finally we set the initial atomic positions.

In [ ]:
# Create a system. Here we define some forcefield settings such as the nonbonded method
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)

# Define the integrator. The Langevin integrator is also a thermostat
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

# Create the Simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)



## Local energy minimization

It is a good idea to run local energy minimization at the start of a simulation because the coordinates in starting configuration file might produce very large forces. This minimization step should take ~1 minute on CPU and a few seconds on GPU.

In [ ]:
print("Minimizing energy")
simulation.minimizeEnergy()

## Setup reporting

To get output from our simulation we can add reporters. We use `DCDReporter` to write the coordinates every 1000 timesteps to 'traj.dcd' and we use `StateDataReporter` to print the timestep, potential energy, temperature, and volume to the screen and to a file called 'md_log.txt'.

In [ ]:
# Write trajectory to a file called traj.dcd every 1000 steps
simulation.reporters.append(DCDReporter('traj.dcd', 1000))

# Print state information to the screen every 1000 steps
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))

# Print the same info to a log file every 100 steps
simulation.reporters.append(StateDataReporter('md_log.txt', 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))


## NVT equilibration
We are using a Langevin integrator which means we are simulating in the NVT ensemble. To equilibrate the temperature we just need to run the simulation for a number of timesteps.

In [ ]:
print('Running NVT')
simulation.step(10000)

## NPT production MD

To run our simulation in the NPT ensemble we need to add in a barostat to control the pressure. We can use MonteCarloBarostat.

In [ ]:
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))

# It is important to call the reinitialize method on the simulation
# otherwise the modifications will not be applied.
simulation.context.reinitialize(preserveState=True)

print('Running NPT')
simulation.step(10000)


## Analysis

We can now do some basic analysis using Python. We will plot the time evolution of the potential energy, temperature, and box volume. Remember that OpenMM itself is primarily an MD engine, for in-depth analysis of your simulations you can use other python packages such as MDtraj, or MDAnalysis.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
data = np.loadtxt('md_log.txt', delimiter=',')

step = data[:,0]
potential_energy = data[:,1]
temperature = data[:,2]
volume = data[:,3]

plt.plot(step, potential_energy)
plt.xlabel("Step")
plt.ylabel("Potential energy (kJ/mol)")
plt.show()
plt.plot(step, temperature)
plt.xlabel("Step")
plt.ylabel("Temperature (K)")
plt.show()
plt.plot(step, volume)
plt.xlabel("Step")
plt.ylabel("Volume (nm^3)")
plt.show()

## Checkpointing

When you run long simulations it is useful to be able to save checkpoints. This means you can restart them in the case of a crash. Or it means you can resume them if you need to fit within the time constraints of a HPC job scheduler.

To run a resume a simulation we need to have three files saved to disk that we can load in:
1. The topology - this will be a PDB file of our solvated system.
2. A serialized `System` -  this is an xml file that contains the forcefield settings.
3. A checkpoint file - this is a binary file that contains the positions, velocities, and box vectors
The first two only need to be saved once, they are constant throughout the simulation. The checkpoint needs to be
saved after a specified number of timesteps.


## Setup the checkpoint

We will create the topology and serialized system files. And we will use `CheckpointReporter` to regularly create checkpoint files.

In [ ]:
# Save the toplogy as a PDB file
with open('topology.pdb', 'w') as output:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(),output)

# save a serialized version of the system. This stores the forcefield parameters
with open('system.xml', 'w') as output:
    output.write(XmlSerializer.serialize(system))

# Setup a checkpoint reporter. This stores the positions, velocities, and box vectors.
simulation.reporters.append(CheckpointReporter('checkpoint.chk', 1000))


## Running for a set time limit

We now run for a set amount of wall clock time (30 seconds) and save checkpoint every 1000 steps

In [ ]:
# run for 30 seconds
simulation.runForClockTime(30.0*seconds)

## Resume from a checkpoint

We now have the required files 'topology.pdb', 'system.xml', and 'checkpoint.chk'. We will need to load them in so we can resume the simulation from the last checkpoint. Note that we have to define the integrator again as well as the simulation reporters. Furthermore we have set the `append=True` flag to the DCD and StateData reporters.

In [ ]:


pdb = PDBFile('topology.pdb')

with open('system.xml') as input:
    system = XmlSerializer.deserialize(input.read())

# Define the integrator.
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

# Create the Simulation
simulation = Simulation(pdb.topology, system, integrator)

# set the positions, velocities, and box vectors from the checkpoint file
simulation.loadCheckpoint('checkpoint.chk')

# We still need to define the reporters again

# Write trajectory to a file called traj.dcd every 1000 steps
simulation.reporters.append(DCDReporter('traj.dcd', 1000, append=True))

# Print state information to the screen every 1000 steps
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))

# Print the same info to a log file every 100 steps
simulation.reporters.append(StateDataReporter('md_log.txt', 100, step=True,
        potentialEnergy=True, temperature=True, volume=True, append=True))


# Setup a checkpoint reporter. This stores the positions, velocities, and box vectors.
simulation.reporters.append(CheckpointReporter('checkpoint.chk', 1000))

# run for 30 seconds

FIXME
# write the code to run for 30seconds of wall clock time
# simulation....



## Resume multiple times

We can emulate doing multiple resumes, i.e. you would do this when you have to adhere to the time limits of a HPC
job scheduler.

In [ ]:
for i in range(3):
    print("Resuming from checkpoint iteration = ", i)

    pdb = PDBFile('topology.pdb')

    with open('system.xml') as input:
        system = XmlSerializer.deserialize(input.read())

    # Define the integrator.
    integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

    # Create the Simulation

    FIXME
    # write the code to create the simulation object
    simulation = Simulation( ... )

    # set the positions, velocities, and box vectors from the checkpoint file
    simulation.loadCheckpoint('checkpoint.chk')

    # We still need to define the reporters again

    # Write trajectory to a file called traj.dcd every 1000 steps
    simulation.reporters.append(DCDReporter('traj.dcd', 1000, append=True))

    # Print state information to the screen every 1000 steps
    simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
            potentialEnergy=True, temperature=True, volume=True))

    # Print the same info to a log file every 100 steps
    simulation.reporters.append(StateDataReporter('md_log.txt', 100, step=True,
        potentialEnergy=True, temperature=True, volume=True, append=True))

    # Setup a checkpoint reporter. This stores the positions, velocities, and box vectors.
    simulation.reporters.append(CheckpointReporter('checkpoint.chk', 1000))


    # run for 30 seconds
    simulation.runForClockTime(30.0*seconds)


## Analysis

In [ ]:
# we can redo the analysis on the longer trajectory

data = np.loadtxt('md_log.txt', delimiter=',')

step = data[:,0]
potential_energy = data[:,1]
temperature = data[:,2]
volume = data[:,3]

plt.plot(step, potential_energy)
plt.xlabel("Step")
plt.ylabel("Potential energy (kJ/mol)")
plt.show()
plt.plot(step, temperature)
plt.xlabel("Step")
plt.ylabel("Temperature (K)")
plt.show()
plt.plot(step, volume)
plt.xlabel("Step")
plt.ylabel("Volume (nm^3)")
plt.show()



## Visualization

We can use the `nglview` package to view the simulation structures and trajectories in the juyter notebook.


For more serious visualization and rendering there a variety of programs available (https://en.wikipedia.org/wiki/List_of_molecular_graphics_systems). The most popular ones are VMD and PyMol.

**Note this does not currently work in Colab**

In [ ]:
!mamba install -y -c conda-forge nglview mdtraj

In [ ]:
import mdtraj
import nglview

traj = mdtraj.load("traj.dcd", top="topology.pdb")
view = nglview.show_mdtraj(traj)
view.add_representation('licorice',selection="water")
view

## Next
Now go to part 2 notebook.